In [1]:
import pandas as pd
import numpy as np

class SME():
    
    def __init__(self):
        self.asked = 0
        self.df = self.get()
        
    def get(self):
        X_train = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/final/employee_departure_dataset_X.csv')
        y_train = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/final/employee_departure_dataset_y.csv')

        X_train['Left'] = y_train
        return X_train
    
    def ask(self, valuedict):
        self.asked += 1
        #  print(self.asked)
        
        if self.asked > 500:
            raise Exception("Sorry, you have asked enough")

        arr = []
        for prop in valuedict:
            val = valuedict[prop]

            if val is None: 
                continue

            val = f"'{val}'" if isinstance(val,str) else val
            # print(prop, val)
            arr.append(f'{prop} == {val}')

            query = ' and '.join(arr)
        result = self.df.query(query)

        if len(result) == 0:
            raise Exception("I don't know")
        
        return (int)(result['Left'].mean())

In [2]:
sme = SME()

In [3]:
sme.df.columns

Index(['RecordId', 'Gender', 'Distance', 'YearsWorked', 'TrainingHours',
       'WorkLifeBalance', 'NumOfProjects', 'JobInvolvement', 'TeamSize',
       'MentorshipReceived', 'TechSkillLevel', 'AttendanceRate', 'StressLevel',
       'PeerFeedbackScore', 'AnnualLeaveDays', 'Certifications',
       'SkillDevelopmentCourses', 'ProjectComplexity', 'WorkSatisfactionScore',
       'JobEngagementScore', 'PhysicalActivityScore', 'MentalWellbeingScore',
       'DepartmentCode', 'PreviousSalary', 'Salary', 'SelfReview',
       'SupervisorReview', 'Left'],
      dtype='object')

In [4]:
X_train = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/final/employee_departure_dataset_X.csv')

In [5]:

sme.df['SelfReview'] = sme.df['SelfReview'].fillna(sme.df['SelfReview'].median())
sme.df['SupervisorReview'] = sme.df['SupervisorReview'].fillna(sme.df['SupervisorReview'].median())
sme.df[['Salary', 'PreviousSalary']] = sme.df[['Salary', 'PreviousSalary']].map(lambda x: x.replace('K', '000') if isinstance(x, str) else x)
sme.df[['Salary', 'PreviousSalary']] = sme.df[['Salary', 'PreviousSalary']].astype(int)

sme.df['Raise'] = (sme.df['Salary'] / sme.df['PreviousSalary']) - 1
sme.df['ReviewDiff'] = sme.df['SelfReview'] - sme.df['SupervisorReview']
sme.df['ReviewOverRaise'] = sme.df['SelfReview'] / (sme.df['Raise'] +.0001)

sme.df['LongDist'] = np.where(sme.df['Distance'].isin(['~20miles', '>30miles']), 1, 0)

sme.df['HighStress'] = np.where(sme.df['StressLevel'] > 3.5, 1, 0)

num = ['HighStress', 'LongDist','Gender', 'SupervisorReview','SelfReview', 'Raise', 'Salary','ReviewOverRaise', 'PreviousSalary'] #, 'LongDist', ReviewOverRaise','ReviewDiff', 'StressLevel'
cat = ['DepartmentCode']#'Distance',
#num = ['Raise', 'StressLevel']

"""
num = ['HighStress', 'LongDist','Gender','RecordId', 'YearsWorked', 'TrainingHours', 
       'WorkLifeBalance', 'NumOfProjects', 'JobInvolvement', 'TeamSize',
       'MentorshipReceived', 'TechSkillLevel', 'AttendanceRate', 'StressLevel',
       'PeerFeedbackScore', 'AnnualLeaveDays', 'Certifications',
       'SkillDevelopmentCourses', 'ProjectComplexity', 'WorkSatisfactionScore',
       'JobEngagementScore', 'PhysicalActivityScore', 'MentalWellbeingScore',
       'PreviousSalary', 'Salary', 'SelfReview',
       'SupervisorReview', 'Raise', 'ReviewDiff', 'ReviewOverRaise']
"""

"\nnum = ['HighStress', 'LongDist','Gender','RecordId', 'YearsWorked', 'TrainingHours', \n       'WorkLifeBalance', 'NumOfProjects', 'JobInvolvement', 'TeamSize',\n       'MentorshipReceived', 'TechSkillLevel', 'AttendanceRate', 'StressLevel',\n       'PeerFeedbackScore', 'AnnualLeaveDays', 'Certifications',\n       'SkillDevelopmentCourses', 'ProjectComplexity', 'WorkSatisfactionScore',\n       'JobEngagementScore', 'PhysicalActivityScore', 'MentalWellbeingScore',\n       'PreviousSalary', 'Salary', 'SelfReview',\n       'SupervisorReview', 'Raise', 'ReviewDiff', 'ReviewOverRaise']\n"

In [6]:
sme.df[num].corr()>.1

,HighStress,LongDist,Gender,SupervisorReview,SelfReview,Raise,Salary,ReviewOverRaise,PreviousSalary
HighStress,True,False,False,False,False,False,False,False,False
LongDist,False,True,False,False,False,False,False,False,False
Gender,False,False,True,False,False,False,False,False,False
SupervisorReview,False,False,False,True,False,False,False,False,False
SelfReview,False,False,False,False,True,False,False,False,False
Raise,False,False,False,False,False,True,False,False,False
Salary,False,False,False,False,False,False,True,False,True
ReviewOverRaise,False,False,False,False,False,False,False,True,False
PreviousSalary,False,False,False,False,False,False,True,False,True


# Code for Feature Engineering

In [8]:
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureEngineering(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        X[['Salary', 'PreviousSalary']] = X[['Salary', 'PreviousSalary']].map(lambda x: x.replace('K', '000') if isinstance(x, str) else x)
        X[['Salary', 'PreviousSalary']] = X[['Salary', 'PreviousSalary']].astype(int)
        X['Raise'] = (X['Salary'] / X['PreviousSalary']) - 1
        X['ReviewDiff'] = X['SelfReview'] - X['SupervisorReview']
        X['ReviewOverRaise'] = X['SelfReview'] / (X['Raise'] +.0001)
        return X

# Establish the preprocessing for numeric and categorical features.

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans

n_clusters = 500

featurepreprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num)#,
        #('cat', OneHotEncoder(), cat)
    ])
preprocessor = Pipeline(steps=[
    #('featureengineering', FeatureEngineering()),
    ('features', featurepreprocessor),
    ('pca', PCA(n_components=.9)),#.9
    ('kmeans', KMeans(n_clusters=n_clusters, random_state=42, n_init='auto'))
    ])
#('kmeans', KMeans(n_clusters=n_clusters, n_init='auto'), num)#+cat)
#('kmeans', KMeans(n_clusters=n_clusters, n_init='auto')

In [11]:
X_dist = preprocessor.fit_transform(sme.df[num+cat])

In [12]:
preprocessor

Pipeline(steps=[('features',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['HighStress', 'LongDist',
                                                   'Gender', 'SupervisorReview',
                                                   'SelfReview', 'Raise',
                                                   'Salary', 'ReviewOverRaise',
                                                   'PreviousSalary'])])),
                ('pca', PCA(n_components=0.9)),
                ('kmeans',
                 KMeans(n_clusters=500, n_init='auto', random_state=42))])

In [13]:
kmeans = KMeans(n_clusters = n_clusters, n_init='auto')
pre = featurepreprocessor.fit_transform(sme.df[num+cat])

In [14]:
X_dist = kmeans.fit_transform(pre)
X_dist

array([[3.9843799 , 3.46131118, 3.11982212, ..., 3.04364651, 3.91318951,
        5.69254439],
       [3.67749992, 2.5490949 , 3.74060957, ..., 3.56119424, 2.37939559,
        4.54013497],
       [3.94477609, 3.5149297 , 2.71983911, ..., 4.99402105, 3.8868003 ,
        5.43206374],
       ...,
       [4.82611894, 4.65486474, 3.57499219, ..., 5.35335405, 4.15179579,
        6.91750257],
       [4.44048989, 1.74061039, 3.67964677, ..., 4.79275392, 4.21130372,
        5.33805316],
       [3.09010936, 4.63962195, 3.96531652, ..., 4.55514935, 3.51427226,
        6.21615922]])

In [15]:
X_dist.shape

(500000, 500)

In [16]:
#How many points are in each cluster?
pd.Series(kmeans.labels_).value_counts()

89     3154
322    3128
56     3038
39     2971
9      2657
       ... 
409     234
499     232
375     225
84      183
466     139
Name: count, Length: 500, dtype: int64

In [17]:
import numpy as np
#What are the reporesentative points? (e.i the ones closest to each centroid)
representative_idx = np.argmin(X_dist, axis=0)
representative_idx.shape

(500,)

In [18]:
#What is the index number of each representative point?
representative_idx

array([  3504,   1059,  14026,   8445,  43188,  18500,   1111,   2103,
          482,    636,  30547,   1595,    292,   8277,    304,   8617,
        15591,   6096,   1933,   9835,    430,   7476,  59025, 102259,
        30635,   2044,   8311,   9517,   7422,  11736,    917,   1385,
        61848,    776,    745,   1655,  33422,    796,   2491,    384,
        98106,   7850,   8120,  12058,  15291,  19967,   8535,   1252,
         4711,  17151,   1643,   1281,   3804,  11948,   1016,   2865,
         2322,   9591,  60361, 103919,   7313,   1119,   7234,  34299,
         4548,  20820,  18048,   9948,   3831,  17859,  14905,  10324,
         1174,  25685,  64834,  24421, 130308,    360,    515,   8108,
        22013,   7699,   1209,  24804, 246712,  26874,   3562,   1186,
        20744,    532,   2847,   7000,    779,   1009, 114276, 104771,
         2731,    825,   2067,  66779,   9933,   5398,   1808,  26127,
         2913,   2111,   4723,  12558,  13850,  22343,   5736,  14123,
      

In [19]:
#What do the reporesentative points look like?
X_representative = sme.df.iloc[representative_idx]
X_representative

,RecordId,Gender,Distance,YearsWorked,TrainingHours,WorkLifeBalance,NumOfProjects,JobInvolvement,TeamSize,MentorshipReceived,...,PreviousSalary,Salary,SelfReview,SupervisorReview,Left,Raise,ReviewDiff,ReviewOverRaise,LongDist,HighStress
3504,3604,0,~10miles,7,24,5,18,3,12,1,...,85000,88000,4.0,5.0,0,0.035294,-1.0,113.013129,0,0
1059,1159,0,<5mile,7,33,5,1,1,8,0,...,60000,61000,4.0,5.0,0,0.016667,-1.0,238.568588,0,0
14026,14126,1,~15miles,8,33,1,9,1,5,1,...,67000,68000,5.0,3.0,0,0.014925,2.0,332.770438,0,0
8445,8545,1,~20miles,7,2,4,10,4,11,1,...,56000,58000,4.0,5.0,0,0.035714,-1.0,111.687276,1,0
43188,43288,1,~20miles,7,84,2,3,1,9,1,...,56000,56000,5.0,4.0,1,0.000000,1.0,50000.000000,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40624,40724,0,>30miles,5,59,3,16,4,13,0,...,59000,61000,3.0,5.0,0,0.033898,-2.0,88.239693,1,1
2282,2382,1,>30miles,9,52,2,5,4,19,0,...,57000,59000,3.0,4.0,0,0.035088,-1.0,85.257018,1,0
29326,29426,1,~20miles,7,37,2,5,3,16,1,...,58000,59000,3.0,4.0,1,0.017241,-1.0,172.996620,1,1
4695,4795,1,~15miles,8,67,1,3,5,5,0,...,58000,60000,4.0,3.0,1,0.034483,1.0,115.664573,0,1


In [20]:
#Look up the leave/stay value of hte 500 representative points
y_representative= []
for i in representative_idx:
    y_representative.append(sme.ask({'RecordId': i+100})) #All record Id's are 100 more than the index value

In [21]:
#Propagate the label value to each point in the cluster.
y_train_propagated = np.empty(len(sme.df), dtype=np.int32)
for i in range(n_clusters):
    y_train_propagated[kmeans.labels_ == i] = y_representative[i]

In [22]:
#Assign the labels and leave/stay values to the DF
sme.df['Labels'] = kmeans.labels_
sme.df['y_pred'] = y_train_propagated

In [23]:
sme.df

,RecordId,Gender,Distance,YearsWorked,TrainingHours,WorkLifeBalance,NumOfProjects,JobInvolvement,TeamSize,MentorshipReceived,...,SelfReview,SupervisorReview,Left,Raise,ReviewDiff,ReviewOverRaise,LongDist,HighStress,Labels,y_pred
0,100,1,>30miles,7,91,4,14,3,8,1,...,4.0,4.0,0,0.013889,0.0,285.941223,1,0,61,0
1,101,0,~10miles,10,52,5,9,2,10,0,...,4.0,4.0,0,0.032258,0.0,123.616788,0,1,448,0
2,102,0,~10miles,10,80,3,13,3,16,0,...,4.0,2.0,1,0.013699,2.0,289.883848,0,0,271,0
3,103,0,~15miles,5,6,3,15,5,16,0,...,5.0,5.0,0,0.015873,0.0,313.027924,0,0,39,0
4,104,1,~10miles,10,3,2,13,3,18,0,...,4.0,4.0,0,0.066667,0.0,59.910135,0,0,352,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,500095,1,>30miles,9,80,1,10,3,14,0,...,4.0,5.0,0,0.032787,-1.0,121.629031,1,1,249,0
499996,500096,0,~20miles,7,0,3,2,5,4,1,...,5.0,5.0,0,0.033333,0.0,149.551346,1,1,315,0
499997,500097,0,~20miles,10,8,3,13,3,13,1,...,5.0,2.0,0,0.046154,3.0,108.099119,1,0,434,1
499998,500098,0,~15miles,10,51,3,7,3,5,0,...,5.0,5.0,1,0.018868,0.0,263.602905,0,0,203,0


In [24]:
sme.df['ClusteringOff']= abs(sme.df['y_pred'] - sme.df['Left']) #Which points are incorrectly labeled / is the clustering off for?

In [25]:
#What Percent incorrect is each cluster?
percent_df = sme.df.groupby('Labels')['ClusteringOff'].sum()/sme.df.groupby('Labels')['ClusteringOff'].count()
percent_df
#percent_df[percent_df>.5].count()

Labels
0      0.013730
1      0.068126
2      0.155480
3      0.039116
4      0.490654
         ...   
495    0.092742
496    0.082627
497    0.255446
498    0.647906
499    0.043103
Name: ClusteringOff, Length: 500, dtype: float64

In [26]:
#What is the total accuracy of the model?
1- sme.df['ClusteringOff'].sum() / sme.df['ClusteringOff'].count()

0.7616

In [27]:
#What percentage of "Leaving" predictions did our model get wrong?
sme.df[sme.df['Left']==1]['ClusteringOff'].sum() / sme.df[sme.df['Left']==1]['ClusteringOff'].count()

0.29999833759094763

In [60]:
sme.df[sme.df['Left']==0]['ClusteringOff'].sum() / sme.df[sme.df['Left']==0]['ClusteringOff'].count()

0.20361207865080633

In [29]:
#How many "Stays" should be "leaves?
sme.df[sme.df['Left']==1]['ClusteringOff'].sum()

54138

In [30]:
#How many "leaves" Should be "stays"?
sme.df[sme.df['Left']==0]['ClusteringOff'].sum()

65062

# Cleaning Pipeline to set up for ML